# Simple Token Distribution

A cadCAD sketch exploring an inflationary token model managed and distributed by community contributors.


## Check cadCAD

This cell doesn't do anything, but it does ensure that you know what version of cadCAD you're running. That way if/when you encounter technical difficulties you can tell the community which version of cadCAD you're running. Might save you hours of pain if all you need to do is upgrade to the latest version. 

In [122]:
%pip show cadCAD

Name: cadCAD
Version: 0.4.23
Summary: cadCAD: a differential games based simulation software package for research, validation, and         Computer Aided Design of economic systems
Home-page: https://github.com/cadCAD-org/cadCAD
Author: Joshua E. Jodesty
Author-email: joshua@block.science
License: LICENSE.txt
Location: /usr/local/lib/python3.8/site-packages
Requires: pathos, fn, pandas, funcy
Required-by: cadCAD-diagram
Note: you may need to restart the kernel to use updated packages.


## Import Stuff

These are the libraries you'll need (cadCAD stuff) and that could be useful (python stuff) for your cadCAD model.

In [123]:
# python stuff
import numpy as np
import random as random

# cadCAD stuff
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD import configs
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

## Initial Params

These are the parameters that control the behavior of a system. For example, if you were modeling a Web3 protocol these would be the parameters within a system that token holders could vote to modify. All of the parameters of a protocol that can be modified should be accessible here. That way it's easy to modify and run similations to explore how parameter changes might affect the network.

In [124]:
# the initial state of the cadCAD model
# in a stock and flow model you can think of this as the stocks
genesis_states = {
    # a variable that tracks the total amount of tokens in the system
    'tokens': 1,
    # contributor stats
    'contributors': {
        i: {'balance': 1, 'skill_level': 5, 'impact': 0} for i in range(1)    
    },
    # the pool of tokens to reward contributors
    'treasury': 0,
    # number of tokens to mint to the treasury 
    'weekly_token_minting': 1000,
    # number of new contributors
    'weekly_new_contributors': 3,
    # maximum amount of effort per week
    'maximum_effort': 7,
    # maximum skill_level of any contributor
    'maximum_skill_level': 10
}

# The parameters to run the model
# Often these are found towards the end of the file near the cadCAD engine
# but we're putting them here so that you can easily configure and run 
# the notebook without having to scroll back and forth
sim_config_dict = {
    # timesteps: weekly
    'T': range(52), 
    # parallel runs
    'N': 1,
    # more advanced stuff
    #'M': {}
}

## Policy Functions

Policy functions are like flows in stock and flow diagrams. They modify the inputs to state update functions.

We start policy functions with p_ so that they're easier to keep track of when we put them into state update blocks. 

In [125]:
# Mint tokens on a weekly basis (each timestep)
def p_mint_tokens(params, step, sH, s):
    minted_tokens = s['weekly_token_minting']
    return ({'minted_tokens': minted_tokens})

# Do work to earn tokens
def p_effort(params, step, sH, s):
    effort = random.randint(1, s['maximum_effort'])
    return ({'effort': effort})

# New community contributors
def p_new_community_contributors(params, step, sH, s):
    latest_contributor_key = list(s['contributors'])[-1]
    new_contributors = {
        i: {
            'balance': 0,
            'skill_level': random.randint(1, s['maximum_skill_level'])
        } for i in range(latest_contributor_key + 1, latest_contributor_key + s['weekly_new_contributors'])
    }
    return ({'new_contributors': new_contributors})


## State Update Functions

These functions take in inputs (state variables and policies) and modify the state.

We start policy functions with p_ so that they're easier to keep track of when we put them into state update blocks.

In [126]:
# Update token count based on weekly minting
def s_update_token_count(params, step, sH, s, _input):
    y = 'tokens'
    x = s[y]
    x += _input['minted_tokens']
    return (y, x)

# Put newly minted tokens into the community treasury
def s_mint_to_treasury(params, step, sH, s, _input):
    y = 'treasury'
    x = s[y]
    x += _input['minted_tokens']
    return (y, x)

# Give new contributors accounts
def s_new_contributor_accounts(params, step, sH, s, _input):
    y = 'contributors'
    x = s[y]
    x.update(_input['new_contributors'])
    return (y, x)

# Update community treasury based on contribution rewards
def s_distribute_treasury(params, step, sH, s, _input):
    y = 'treasury'
    x = s[y]
    for i in s['contributors']:
        impact = s['contributors'][i]['skill_level'] * _input['effort']
        if x >= impact:
            x -= impact
        else:
            break
    return (y, x)

# Distribute rewards to contributors
def s_reward_contributions(params, step, sH, s, _input):
    y = 'contributors'
    x = s[y]
    for i in x:
        impact = s['contributors'][i]['skill_level'] * _input['effort']
        if s['treasury'] >= impact:
            x[i]['balance'] += impact
        else:
            break
    return (y, x)


## State Update Block

If you're approaching cadCAD from Web3 you can think of the state of the model as something that evolves in blocks. Each block has a set of actions that updates the state. Those transactions then get batched into blocks to be processed together. In cadCAD blocks are called "`partial_state_update_blocks`." As you can see below, this is an array that is very similar to a "block" in a blockchain in that it represents a set of actions to update the state. That state is then updated across many timesteps. This is similar to how the state of a blockchain is updated over many timesteps as new blocks are added.



In [127]:
partial_state_update_blocks = [
    { 
        # mint tokens
        'policies': { 
            'minted_tokens': p_mint_tokens
        },
        # update token count and community treasury
        'variables': {
            'tokens': s_update_token_count,
            'treasury': s_mint_to_treasury
        }
    },
    {
        # add new contributors
        'policies': {
            'new_contributors': p_new_community_contributors
        },
        # give contributors accounts
        'variables': {
            'contributors': s_new_contributor_accounts
        }
    },
    {
        # do work
        'policies': {
            'effort': p_effort
        },
        # recognize and reward contributors
        'variables': {
            'treasury': s_distribute_treasury,
            'contributors': s_reward_contributions
        }
    }
    # TO DO: 
    # Add in intermediate step between doing work and getting rewarded
    # At this step contributors would earn impact points
    # Then in the reward stage the weekly rewards would be distributed between contributors based on impact points
    # 20% of rewards would stay in the treasury for long-term projects, and 80% would go to weekly contributors 
]

## Running the cadCAD Engine

In [128]:
# imported some addition utilities to help with configuration set-up
exp = Experiment()
c = config_sim(sim_config_dict)
# The configurations above are then packaged into a `Configuration` object
del configs[:]
# dict containing variable names and initial values
exp.append_configs(initial_state=genesis_states, 
                    # dict containing state update functions 
                    partial_state_update_blocks=partial_state_update_blocks, 
                    # preprocessed dictionaries containing simulation parameters
                    sim_configs=c)


In [129]:
%%capture
exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(exec_mode.local_mode)

# pass the configuration object inside an array
simulation = Executor(exec_context=local_mode_ctx, configs=configs) 
# the `execute()` method returns a tuple; its first elements contains the raw results
raw_system_events, tensor_field, sessions = simulation.execute() 

## Data Visualization

This is often half the battle. Not only do need to design and build a cadCAD model, but you need to understand how it's working and be able to effectively communicate that to other people. A picture says a thousand words, thus enter data viz. Getting good at using python data viz libraries is probably the highest leverage thing you can do after you learn the cadCAD basics.

In [130]:
%matplotlib inline
import pandas as pd
simulation_result = pd.DataFrame(raw_system_events)
simulation_result.set_index(['subset', 'run', 'timestep', 'substep'])

tokens  \
subset run timestep substep           
0      1   0        0             1   
           1        1          1001   
                    2          1001   
                    3          1001   
           2        1          2001   
...                             ...   
           51       2         51001   
                    3         51001   
           52       1         52001   
                    2         52001   
                    3         52001   

                                                                  contributors  \
subset run timestep substep                                                      
0      1   0        0        {0: {'balance': 1, 'skill_level': 5, 'impact':...   
           1        1        {0: {'balance': 1, 'skill_level': 5, 'impact':...   
                    2        {0: {'balance': 1, 'skill_level': 5, 'impact':...   
                    3        {0: {'balance': 16, 'skill_level': 5, 'impact'...   
           2        1        {0: {'balance': 16, 'skill_level': 5, 'impact'...   
...                                                                        ...   
           51       2        {0: {'balance': 1021, 'skill_level': 5, 'impac...   
                    3        {0: {'balance': 1046, 'skill_level': 5, 'impac...   
           52       1        {0: {'balance': 1046, 'skill_level': 5, 'impac...   
                    2        {0: {'balance': 1046, 'skill_level': 5, 'impac...   
                    3        {0: {'balance': 1081, 'skill_level': 5, 'impac...   

                             treasury  weekly_token_minting  \
subset run timestep substep                                   
0      1   0        0               0                  1000   
           1        1            1000                  1000   
                    2            1000                  1000   
                    3             958                  1000   
           2        1            1958                  1000   
...                               ...                   ...   
           51       2            1027                  1000   
                    3              27                  1000   
           52       1            1027                  1000   
                    2            1027                  1000   
                    3               5                  1000   

                             weekly_new_contributors  maximum_effort  \
subset run timestep substep                                            
0      1   0        0                              3               7   
           1        1                              3               7   
                    2                              3               7   
                    3                              3               7   
           2        1                              3               7   
...                                              ...             ...   
           51       2                              3               7   
                    3                              3               7   
           52       1                              3               7   
                    2                              3               7   
                    3                              3               7   

                             maximum_skill_level  simulation  
subset run timestep substep                                   
0      1   0        0                         10           0  
           1        1                         10           0  
                    2                         10           0  
                    3                         10           0  
           2        1                         10           0  
...                                          ...         ...  
           51       2                         10           0  
                    3                         10           0  
           52       1                         10           0  
    

simulation_result.plot('timestep', ['tokens', 'treasury'], grid=True, 
        colormap = 'gist_rainbow',
        xticks=list(simulation_result['timestep'].drop_duplicates()), 
        yticks=list(range(1+(simulation_result['tokens']+simulation_result['treasury']).max())))

In [131]:
raw_system_events

 'skill_level': 10},
   84: {'balance': 93, 'skill_level': 3},
   85: {'balance': 72, 'skill_level': 3},
   86: {'balance': 168, 'skill_level': 7},
   87: {'balance': 154, 'skill_level': 7},
   88: {'balance': 44, 'skill_level': 2},
   89: {'balance': 80, 'skill_level': 4},
   90: {'balance': 140, 'skill_level': 7},
   91: {'balance': 126, 'skill_level': 7},
   92: {'balance': 36, 'skill_level': 2},
   93: {'balance': 28, 'skill_level': 2},
   94: {'balance': 112, 'skill_level': 8},
   95: {'balance': 91, 'skill_level': 7},
   96: {'balance': 130, 'skill_level': 10},
   97: {'balance': 12, 'skill_level': 2},
   98: {'balance': 48, 'skill_level': 8},
   99: {'balance': 40, 'skill_level': 8},
   100: {'balance': 35, 'skill_level': 7},
   101: {'balance': 0, 'skill_level': 4},
   102: {'balance': 0, 'skill_level': 7}},
  'weekly_token_minting': 1000,
  'weekly_new_contributors': 3,
  'maximum_effort': 7,
  'maximum_skill_level': 10,
  'simulation': 0,
  'subset': 0,
  'run': 1,
  'substep